# 1. Population Genetics Simulation

Create a program that simulates the allelic frequency in a finite diploid population for a certain number of generations.

The program takes as input the initial allele frequencies, the fitness of each genotype, the population size, and the number of generations. Because these simulations are stochastic each run of the simulation will give a different result, to allow an idea of the behavior of the allelic frequencies, your program should repeat the simulations many times for each parameter set and plot all the results in a single graph. The number of simulations should also be determined by the user. You can start your program using the variable definitions in the cell below.

Your program should output two graphs. The first should show the allele frequency at each generation, and the other should be a histogram with the final values of the allele frequency. Something like this:

![simulation](Sim1.png)

![histogram](Sim2.png)

Last year a student used this homework as the starting point for her project to create a population genetics simulator for BIOL040. You can see the final project here: http://dna.pomona.edu:5006/pop_gen_sim

In [1]:
import numpy as np
import plotly.graph_objs as go


# Function to simulate one generation
def simulate_generation(pop, fAfA, fAfa, fafa, wAA, wAa, waa):
    # Calculate the weighted frequencies for selection
    weighted_freqs = [fAfA * wAA, fAfa * wAa, fafa * waa]
    weighted_freqs /= np.sum(weighted_freqs)
    
    # Translate into allele frequencies for A and a
    pA = weighted_freqs[0] + weighted_freqs[1] / 2
    pa = weighted_freqs[2] + weighted_freqs[1] / 2
    
    # Simulate the next generation's allele frequencies
    offspring = np.random.choice(['AA', 'Aa', 'aa'], size=pop, p=weighted_freqs)
    fAfA = np.sum(offspring == 'AA') / pop
    fAfa = np.sum(offspring == 'Aa') / pop
    fafa = np.sum(offspring == 'aa') / pop
    
    return pA, pa, fAfA, fAfa, fafa


# Main simulation function
def simulate_allele_frequencies(initA, inita, wAA, wAa, waa, pop, gen, sim):
    all_frequencies = []
    final_frequencies = []

    # Runs desired num of simulations
    for _ in range(sim):
        fAfA = initA ** 2
        fAfa = 2 * (initA * inita)
        fafa = inita ** 2
        freq = []

        # Runs gen num of generations for each simulation
        for _ in range(gen):
            pA, pa, new_fAfA, new_fAfa, new_fafa = simulate_generation(pop, fAfA, fAfa, fafa, wAA, wAa, waa)
            fAfA = new_fAfA
            fAfa = new_fAfa
            fafa = new_fafa
            
            # Tracks for Allele A - (Can change for Allele a if desired)
            freq.append(pA)
        
        all_frequencies.append(freq)
        final_frequencies.append(freq[-1])
    
    return all_frequencies, final_frequencies


# Plotting functions using plotly.graph_objs
def plot_allele_frequencies(all_frequencies, gen):
    layout = go.Layout(
        title="Allele Frequencies Over Generations",
        xaxis=dict(
            title="Generation"
        ),
        yaxis=dict(
            title="Frequency of Allele A"
        ),
        showlegend=True
    )

    fig = go.Figure(layout=layout)

    # Show each simulation with a line
    for frequencies in all_frequencies:
        fig.add_trace(go.Scatter(x=list(range(gen)), y=frequencies, mode='lines', line=dict(width=1), opacity=0.8))
    fig.show()

def plot_final_frequencies_histogram(final_frequencies):
    layout = go.Layout(
        title="Distribution of Final Allele Frequencies",
        xaxis=dict(
            title="Final Frequency of Allele A"
        ),
        yaxis=dict(
            title="Count"
        )
    )

    # Show histogram of Final Allele Frequencies
    fig = go.Figure(data=[go.Histogram(x=final_frequencies, nbinsx=20, marker=dict(color='blue'), opacity=0.75)], layout=layout)
    fig.show()


def main():
    #Allele frequencies
    initA = 0.50
    inita = 0.50

    #Fitnesses
    wAA = 1  # fAfA
    wAa = 1  # fAfa
    waa = 1  # fafa

    #Pop Size
    pop = 1000

    #Number of generations
    gen = 100

    #Number of simulations
    sim = 100

    # Run simulations
    all_frequencies, final_frequencies = simulate_allele_frequencies(initA, inita, wAA, wAa, waa, pop, gen, sim)

    # Plot results using Plotly
    plot_allele_frequencies(all_frequencies, gen)
    plot_final_frequencies_histogram(final_frequencies)


main()